Data from:

Physician & Other Supplier Payments - Detailed Data
https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/Medicare-Provider-Charge-Data/Physician-and-Other-Supplier2017

Hospital Outpatient - Detailed Data
https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/Medicare-Provider-Charge-Data/Outpatient

APC to CPT/HCPCS crosswalk - Addendum B – January 2020
https://www.cms.gov/Medicare/Medicare-Fee-for-Service-Payment/HospitalOutpatientPPS/Addendum-A-and-Addendum-B-Updates

Zip Code to CBSA
https://www.huduser.gov/portal/datasets/usps_crosswalk.html![image.png](attachment:image.png)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
%matplotlib inline

In [3]:
pd.options.display.max_columns = 25

In [4]:
pd.options.display.max_rows = 25

In [5]:
# Reading in the CBSA to ZIP code crosswalk

cbsa_to_zip = pd.read_excel('CBSA_ZIP_032020.xlsx', 
                            usecols = ['CBSA', 'ZIP'],
                            dtypes={'ZIP':'str'})

In [6]:
#Reading in HCPCS to APC crosswalk with Short Descriptor column

hcpcs_to_apc = pd.read_excel('2020_january_web_addendum_b.12312019.xlsx', header = 2, usecols = ['HCPCS Code', 'Short Descriptor', 'APC '])

In [7]:
#Reading in the Physician & Other Supplier Payments while discarding some columns

phys_other_payments = pd.read_csv('Medicare_Provider_Util_Payment_PUF_CY2017.txt', 
    sep='\t', 
    skiprows = [1], 
    usecols = lambda column : column not in ['average_submitted_chrg_amt', 'average_Medicare_payment_amt', 'average_Medicare_standard_amt', 'hcpcs_drug_indicator', 'medicare_participation_indicator'], 
    dtype = {'nppes_provider_zip':'str'},
    low_memory=False)

In [8]:
# Reading in the Hospital Outpatient while discarding some columns

column_exclude_list = ['Outlier\nComprehensive\nAPC\nServices', 'Average\nMedicare\nOutlier\nAmount', 'Average\nEstimated\nTotal\nSubmitted\nCharges', 'Average\nMedicare\nPayment\nAmount']

hosp_payments = pd.read_excel('MUP_OHP_R19_P04_V10_D17_APC_Provider.xlsx', 
                              header = 5, 
                              usecols = lambda column : column not in column_exclude_list)

In [9]:
hcpcs_to_apc.columns = ['hcpcs', 'descriptor', 'apc']

In [10]:
cbsa_to_zip.columns = ['cbsa', 'zip']

In [11]:
phys_other_payments.columns = ['npi', 'last_org_name', 'first_name', 'mi', 'creds', 'gender', 'entity', 'street1', 'street2' , 'city', 'zip','state', 'country', 'provider_type', 'service_loc', 'hcpcs', 'description', 'services_cnt', 'benefic_cnt', 'benefic_d_cnt', 'avg_medi_allowed_amt']

In [12]:
hosp_payments.columns = ['provider_id', 'provider_name', 'street', 'city', 'state', 'zip', 'region', 'apc', 'description', 'benefic_cnt', 'services_cnt', 'avg_medi_allowed_amt']

In [13]:
# Dropping non US

phys_other_payments = phys_other_payments[phys_other_payments.country == 'US']

In [14]:
# Making a dataframe with only office entities to compare with the Hospital Data.

phys_other_payments = phys_other_payments[phys_other_payments['entity'] == 'O']

In [15]:
phys_other_payments = phys_other_payments[phys_other_payments['provider_type'] == 'Ambulatory Surgical Center']

In [16]:
phys_other_payments['zip'] = phys_other_payments['zip'].str[:5]

In [17]:
# Putting leading zero back on zips

cbsa_to_zip['zip'] = cbsa_to_zip['zip'].apply(lambda x: '{0:0>5}'.format(x))

In [18]:
# Merging CBSA to Zip in physcian billing data

phys_asc_cbsa = pd.merge(phys_other_payments, cbsa_to_zip, how='left', on = ['zip'])

del phys_other_payments

In [19]:
# Dropping NaN values in the apc column of hcpcs to apc crosswalk

hcpcs_to_apc = hcpcs_to_apc.dropna(subset=['apc'])

In [20]:
# Putting leading zero back on zips

hosp_payments['zip'] = hosp_payments['zip'].apply(lambda x: '{0:0>5}'.format(x))

In [21]:
# Adding matching APC code to HCPCS in the office dataframe, keeping only rows that have an apc match

phys_asc_apc = pd.merge(phys_asc_cbsa, hcpcs_to_apc, how = 'inner', on = ['hcpcs'])

del phys_asc_cbsa

In [22]:
phys_apc_data = phys_asc_apc.drop(['first_name', 'mi', 'creds', 'gender'], axis = 1)

del phys_asc_apc

In [23]:
# Adding CBSA column to hospital data, keeping only rows with a CBSA

hosp_payments_cbsa = pd.merge(hosp_payments, cbsa_to_zip, how = 'inner', on = ['zip'])

del hosp_payments

In [24]:
cbsa_to_region = hosp_payments_cbsa.loc[:,['region', 'cbsa']]

In [25]:
cbsa_to_region = cbsa_to_region.drop_duplicates(subset=['cbsa'])

In [26]:
apc_to_description = hosp_payments_cbsa.loc[:, ['apc', 'description']]

In [27]:
apc_to_description = apc_to_description.drop_duplicates(subset=['apc'])

In [28]:
list = []

for cbsa in hosp_payments_cbsa['cbsa'].unique():
    for apc in hosp_payments_cbsa['apc'].unique():
        selection = hosp_payments_cbsa[(hosp_payments_cbsa['cbsa'] == cbsa) & (hosp_payments_cbsa['apc'] == apc)]
        max_value = round(selection.avg_medi_allowed_amt.max(), 2)
        avg_value = round(selection.avg_medi_allowed_amt.mean(), 2)
        min_value = round(selection.avg_medi_allowed_amt.min(), 2)
        list.append([cbsa, apc, max_value, avg_value, min_value])

In [29]:
list_df = pd.DataFrame(list)

del list

In [30]:
list_df.columns = ['cbsa','apc', 'hosp_max', 'hosp_avg', 'hosp_min']

In [31]:
list_df_region = pd.merge(list_df, cbsa_to_region, how='left', on = ['cbsa'])

del list_df

In [32]:
hosp_table = pd.merge(list_df_region, apc_to_description, how='left', on=['apc'])

del list_df_region

In [33]:
hosp_table = hosp_table.dropna(subset=['hosp_avg'])

In [34]:
phys_apc_data = phys_apc_data.dropna(subset=['cbsa'])

In [35]:
phys_apc_data['cbsa'] = phys_apc_data['cbsa'].astype('int')

In [36]:
phys_apc_data['apc'] = phys_apc_data['apc'].astype('int')

In [37]:
# Getting all the unique apc's that are in the hospital table

hosp_apc_list = hosp_table['apc'].unique()

In [38]:
# Narrowing the Ambulatory Surgical Center table down to rows that have an apc that is in the hosp table

phys_apc_data = phys_apc_data[phys_apc_data['apc'].isin(hosp_apc_list)]

In [39]:
phys_apc_data = phys_apc_data[phys_apc_data['cbsa'].isin(hosp_table['cbsa'].unique())]

In [40]:
phys_testing = phys_apc_data.loc[:, ['hcpcs', 'avg_medi_allowed_amt', 'cbsa']]

In [41]:
list2 = []

for cbsa in phys_testing['cbsa'].unique():
    for hcpcs in phys_testing['hcpcs'].unique():
        selection = phys_testing[(phys_testing['cbsa'] == cbsa) & (phys_testing['hcpcs'] == hcpcs)]
        max_value = round(selection.avg_medi_allowed_amt.max(), 2)
        avg_value = round(selection.avg_medi_allowed_amt.mean(), 2)
        min_value = round(selection.avg_medi_allowed_amt.min(), 2)
        list2.append([cbsa, hcpcs, max_value, avg_value, min_value])

In [42]:
list2_df = pd.DataFrame(list2)

del list2

In [43]:
list2_df.columns = ['cbsa','hcpcs', 'office_max', 'office_avg', 'office_min']

In [44]:
list2_df

,cbsa,hcpcs,office_max,office_avg,office_min
0,46700,0191T,3549.09,3295.66,3042.24
1,46700,29823,837.13,837.13,837.13
2,46700,29824,1231.07,1123.79,1016.51
3,46700,29827,3391.40,3313.28,3235.16
4,46700,29880,1674.25,1657.51,1640.77
...,...,...,...,...,...
347163,47080,60240,NaN,NaN,NaN
347164,47080,25075,NaN,NaN,NaN
347165,47080,29845,NaN,NaN,NaN
347166,47080,64910,NaN,NaN,NaN


In [45]:
list2_df = list2_df.dropna(subset=['office_avg'])

In [50]:
hosp_office_data = pd.merge(list2_df, hosp_table, how='inner', on = ['cbsa'])



In [49]:
del list2_df
del hosp_table
del phys_apc_data
hosp_table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23354 entries, 0 to 48993
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   cbsa         23354 non-null  int64  
 1   apc          23354 non-null  int64  
 2   hosp_max     23354 non-null  float64
 3   hosp_avg     23354 non-null  float64
 4   hosp_min     23354 non-null  float64
 5   region       23354 non-null  object 
 6   description  23354 non-null  object 
dtypes: float64(3), int64(2), object(2)
memory usage: 1.4+ MB


In [51]:
hosp_office_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 670031 entries, 0 to 670030
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   cbsa         670031 non-null  int64  
 1   hcpcs        670031 non-null  object 
 2   office_max   670031 non-null  float64
 3   office_avg   670031 non-null  float64
 4   office_min   670031 non-null  float64
 5   apc          670031 non-null  int64  
 6   hosp_max     670031 non-null  float64
 7   hosp_avg     670031 non-null  float64
 8   hosp_min     670031 non-null  float64
 9   region       670031 non-null  object 
 10  description  670031 non-null  object 
dtypes: float64(6), int64(2), object(3)
memory usage: 61.3+ MB


In [56]:
# Selcting a cbsa and a apc and returing the max, mean, and min values for those selections.

selection = hosp_payments_cbsa[(hosp_payments_cbsa['cbsa'] == 20020) & (hosp_payments_cbsa['apc'] == 5072)]
max_value = round(selection.avg_medi_allowed_amt.max(), 2)
avg_value = round(selection.avg_medi_allowed_amt.mean(), 2)
min_value = round(selection.avg_medi_allowed_amt.min(), 2)

print('Region = ', selection['region'][0])
print('Procedure = ', selection['description'][0])
print()
print('Max = ', max_value)
print('Mean = ', avg_value)
print('Min = ', min_value)

Region =  AL - Dothan
Procedure =  Level 2 Excision/ Biopsy/ Incision and Drainage

Max =  1102.92
Mean =  1028.73
Min =  986.77
